# VGG (Networks Using Blocks)


In [3]:
import torch
from torch import nn
from d2l import torch as d2l

## Model

### VGG Blocks

A VGG block consists of a sequence of convolutions with  3×3  kernels with padding of 1 (keeping height and width) followed by a  2×2  max-pooling layer with stride of 2 (halving height and width after each block). 

In [9]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers) ###

### VGG Network

In [10]:
# 5 VGG blocks
# why 5 blocks? since each VGG block halve height and width after each block, so 224 / (2**5) = 7
# VGG-11: 8 convolutional layers + 3 fully connected layers
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512)) 

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(
        *conv_blks, nn.Flatten(), ####
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(p=0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

View the model

In [11]:
X = torch.rand(size=(1, 1, 224, 224), dtype=torch.float32)
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape: \t', X.shape)

Sequential output shape: 	 torch.Size([1, 64, 112, 112])
Sequential output shape: 	 torch.Size([1, 128, 56, 56])
Sequential output shape: 	 torch.Size([1, 256, 28, 28])
Sequential output shape: 	 torch.Size([1, 512, 14, 14])
Sequential output shape: 	 torch.Size([1, 512, 7, 7])
Flatten output shape: 	 torch.Size([1, 25088])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output shape: 	 torch.Size([1, 10])


512 * 7 * 7 = 25088

## Training

Test on the Fashion-MNIST dataset.

**Since VGG-11 is computationally more demanding than AlexNet
we construct a network with a smaller number of channels.**
This is more than sufficient for training on Fashion-MNIST.
The **model training** process is similar to that of AlexNet. 
Again observe the close match between validation and training loss, 
suggesting only a small amount of overfitting.


In [14]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())